# Instalação de Dependências

O framework utilizado para trabalhar com as imagens foi o scikit-image.

Além disso, para facilitar a visualização e teste dos resultados a biblioteca ipywidgets também foi usada.

In [1]:
%pip install scikit-image
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [71]:
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from ipywidgets import interact, interact_manual

# Ajuste de imagem

Para que o algoritmo funcione melhor é útil cortar a imagem apenas na região onde o cubo aparece.
Ajustar o contraste também pode ser útil para ressaltar bordas dependendo do tipo de cubo utilizado ou do ambiente onde a foto foi tirada.

In [158]:
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.exposure import rescale_intensity


loaded_image = skimage.io.imread("test_images/cube_5.jpg")
cube = np.zeros_like(loaded_image)
grey_cube = rgb2gray(cube)
borders = np.zeros_like(loaded_image)
labels = np.zeros_like(loaded_image)


def crop_image(image, size_factor, dx=0, dy=0):
    w, h, *_ = loaded_image.shape
    size = min(w, h)
    bounding_box = (
        int((w - size * size_factor) / 2 + dy * size * (1 - size_factor)),
        int((w + size * size_factor) / 2 + dy * size * (1 - size_factor)),
        int((h - size * size_factor) / 2 - dx * size * (1 - size_factor)),
        int((h + size * size_factor) / 2 - dx * size * (1 - size_factor)),
    )

    cropped = image[
        bounding_box[0] : bounding_box[1],
        bounding_box[2] : bounding_box[3],
    ]
    cropped = resize(cropped, (512, 512), anti_aliasing=True)
    return cropped

def adjust_contrast(image, low, high):
    p_range = np.percentile(image, (low, high))
    return rescale_intensity(image, in_range=tuple(p_range))

def adjust_image(size_factor=1, dx=0, dy=0, low_percentil=2, high_percentil=98):
    global cube
    global grey_cube

    cube = loaded_image.copy()
    cube = crop_image(cube, size_factor, dx, dy)
    cube = adjust_contrast(cube, low_percentil, high_percentil)
    grey_cube = 1 - rgb2gray(cube)

    plt.imshow(cube)
    plt.show()

interact(adjust_image, 
         size_factor=(0.1, 1, 0.01), 
         dx = (-1, 1, 0.01),
         dy = (-1, 1, 0.01),
         low_percentil = (0, 100),
         high_percentil = (0, 100),
)

interactive(children=(FloatSlider(value=1.0, description='size_factor', max=1.0, min=0.1, step=0.01), FloatSli…

<function __main__.adjust_image(size_factor=1, dx=0, dy=0, low_percentil=2, high_percentil=98)>

# Extração de bordas

O primeiro passo do método é extrair as bordas da imagem utilizando o algoritmo Canny.
Neste passo é possível contornar algumas falhas na extração de borda através do algoritmo de fechamento.

In [180]:
from skimage.feature import canny
from skimage.morphology import binary_dilation, binary_closing, thin
from skimage.morphology import disk, star


def extract_borders(image, sigma, closing):
    borders = canny(image, sigma, low_threshold=0)
    closing = binary_closing(borders, star(closing))
    skeleton = thin(closing)
    dilated = binary_dilation(skeleton, disk(2))
    return dilated

def show_borders(sigma=1, closing=1):
    global grey_cube
    global borders

    borders = extract_borders(grey_cube, sigma, closing)
    plt.imshow(cube)
    plt.imshow(borders, alpha=(borders == 1) * 0.9)
    plt.show()

interact(show_borders, sigma=(0, 5, 0.1), closing=(1, 10))

interactive(children=(FloatSlider(value=1.0, description='sigma', max=5.0), IntSlider(value=1, description='cl…

<function __main__.show_borders(sigma=1, closing=1)>

# Extração dos quadrados

Para extrair os quadrados utilizamos as bordas detectadas no passo anterior e separamos cada área delimitada por elas.

Nem todas as regiões extraidas são de nosso interesse, por isso filtramos as regiões desejadas de acordo com:
- A área ocupada pela região;
- A solidez da região.

Neste momento espera-se que a imagem resultante contenha apenas 27 regiões. 9 Para cada uma das faces visíveis.

In [186]:
from skimage import measure
from skimage.color import label2rgb
from skimage.morphology import convex_hull_image


def extract_labels(borders, min_area, max_area, min_solidity):
    labels, num_labels = ndimage.label(borders == 0)
    props = measure.regionprops(labels)

    for i in range(1, num_labels+1):
        current_label = (labels == i)
        area = props[i - 1].area
        solidity = props[i - 1].solidity

        if solidity < min_solidity:
            labels[current_label] = 0

        elif not (min_area**2 < area < max_area**2):
            labels[current_label] = 0

    labels, num_labels = ndimage.label(labels)
    return labels, num_labels

def show_labels(min_area=20, max_area=512, min_solidity=0.8):
    global cube
    global borders
    global labels
    
    labels, num_labels = extract_labels(borders, min_area, max_area, min_solidity)

    print(f"Partes encontradas: {num_labels}/27")

    plt.imshow(label2rgb(labels, cube, kind="avg"))
    plt.show()

interact(show_labels, min_area=(0, 100), max_area=(0, 512), min_solidity=(0, 1, 0.1))

interactive(children=(IntSlider(value=20, description='min_area'), IntSlider(value=512, description='max_area'…

<function __main__.show_labels(min_area=20, max_area=512, min_solidity=0.8)>